# Tutorial 1: Features & Visibility

**Goal:** Understand HERON's core design and create FeatureProviders that define **what** agents observe and **who** can see it.

**Time:** ~15 minutes

---

## Why HERON?

Training RL agents for multi-agent systems (power grids, traffic, robotics) requires solving:

| Challenge | Traditional Approach | HERON Solution |
|-----------|---------------------|----------------|
| Partial observability | Manual filtering code per agent | Declarative `visibility: ClassVar = ['owner', 'upper_level']` |
| Hierarchical control | Hardcoded parent-child logic | Built-in `FieldAgent` → `CoordinatorAgent` → `SystemAgent` |
| Realistic timing | Single execution mode | Dual modes: sync (training) + event-driven (testing) |
| Swappable coordination | Rewrite code per protocol | Pluggable `SetpointProtocol`, `PriceSignalProtocol` |

### The Key Insight: Agent-Centric Design

**Traditional (environment-centric):** Environment manages everything, agents are stateless policy wrappers.

**HERON (agent-centric):** Agents are first-class citizens with state, timing, and hierarchy. Features declare their own visibility.

```python
# Traditional: 50+ lines of manual filtering
def get_obs_for_battery(global_state, agent_id):
    if agent_id == 'battery_1': return global_state['battery_1']['soc']
    elif agent_id == 'microgrid_1': return global_state['battery_1']['soc']
    # ... more filtering logic

# HERON: 3 lines - visibility is declarative
@dataclass(slots=True)
class BatterySOC(FeatureProvider):
    visibility: ClassVar[Sequence[str]] = ['owner', 'upper_level']  # Done!
    soc: float = 0.5
```

---

## The Key Idea: Declarative Visibility

With HERON, visibility is **declared**, not coded:

```python
@dataclass(slots=True)
class BatterySOC(FeatureProvider):
    visibility: ClassVar[Sequence[str]] = ['owner', 'upper_level']  # Battery + coordinator can see
    soc: float = 0.5
```

### Visibility Levels

| Level | Who Can See | Example Use |
|-------|------------|-------------|
| `public` | Everyone | Time of day, market prices |
| `owner` | The owning agent | Internal device state |
| `upper_level` | Owner + parent in hierarchy | Battery SOC visible to microgrid |
| `system` | System operator (top level) | Grid-wide metrics |

## The FeatureProvider Interface

HERON's `FeatureProvider` base class provides sensible defaults. You only need to:

1. Use `@dataclass(slots=True)` decorator
2. Declare `visibility` as a `ClassVar[Sequence[str]]`
3. Define your data fields with defaults

```python
from dataclasses import dataclass
from typing import ClassVar, Sequence
from heron.core.feature import FeatureProvider

@dataclass(slots=True)
class MyFeature(FeatureProvider):
    visibility: ClassVar[Sequence[str]] = ['owner', 'upper_level']
    
    value: float = 0.0  # Your data field
```

### What You Get for Free

The base class automatically provides:
- `vector()` → Returns numpy array of all fields
- `names()` → Returns list of field names
- `to_dict()` / `from_dict()` → Serialization
- `set_values(**kwargs)` → Update field values
- `reset(**overrides)` → Reset to defaults
- `is_observable_by(...)` → Visibility checking
- `feature_name` → Auto-set to class name (can be customized)

### When to Override

Override methods only when you need custom behavior:
- `vector()` → Custom normalization or computed features
- `set_values()` → Validation or clamping
- `names()` → Custom feature names for debugging

Now let's build some real features.

## Example 1: Battery State of Charge (Simple)

**Visibility:** Owner (battery) + upper_level (microgrid coordinator)

This example uses the default `vector()` implementation.

In [1]:
from dataclasses import dataclass
from typing import ClassVar, Sequence
import numpy as np

from heron.core.feature import FeatureProvider


@dataclass(slots=True)
class BatterySOC(FeatureProvider):
    """Battery state of charge. Visible to owner and coordinator.
    
    Uses default vector() - returns [soc] as numpy array.
    """
    visibility: ClassVar[Sequence[str]] = ['owner', 'upper_level']
    
    soc: float = 0.5  # State of charge [0, 1]
    
    def set_values(self, **kw):
        """Override to add validation/clamping."""
        if 'soc' in kw:
            self.soc = float(np.clip(kw['soc'], 0.0, 1.0))


# Test it
battery_soc = BatterySOC(soc=0.75)
print(f"Vector: {battery_soc.vector()}")
print(f"Names: {battery_soc.names()}")
print(f"Visibility: {battery_soc.visibility}")
print(f"Feature name: {battery_soc.feature_name}")  # Auto-set to class name

Vector: [0.75]
Names: ['soc']
Visibility: ['owner', 'upper_level']
Feature name: BatterySOC


## Example 2: Generator Output (Custom vector)

**Visibility:** Owner + upper_level + system (grid operator needs to see total generation)

This example overrides `vector()` to return normalized output.

In [2]:
@dataclass(slots=True)
class GenOutput(FeatureProvider):
    """Generator power output. Visible to owner, coordinator, AND system.
    
    Overrides vector() to return normalized value for ML training.
    """
    visibility: ClassVar[Sequence[str]] = ['owner', 'upper_level', 'system']
    
    p_mw: float = 0.0   # Power output [MW]
    p_max: float = 5.0  # Maximum capacity [MW]
    
    def vector(self) -> np.ndarray:
        """Return normalized power for better ML training."""
        p_norm = self.p_mw / max(self.p_max, 1e-6)
        return np.array([p_norm], dtype=np.float32)
    
    def names(self):
        """Custom name reflecting the normalization."""
        return ['p_normalized']


# Test it
gen = GenOutput(p_mw=2.5, p_max=5.0)
print(f"Vector (normalized): {gen.vector()}")  # [0.5]
print(f"Raw p_mw: {gen.p_mw} MW")
print(f"to_dict(): {gen.to_dict()}")  # Still serializes raw values

Vector (normalized): [0.5]
Raw p_mw: 2.5 MW
to_dict(): {'p_mw': 2.5, 'p_max': 5.0}


## Example 3: Public Information (Cyclic Encoding)

**Visibility:** `public` — everyone sees this (time of day, market prices)

In [3]:
@dataclass(slots=True)
class TimeOfDay(FeatureProvider):
    """Time feature - visible to everyone.
    
    Uses cyclic encoding so hour 23 is close to hour 0.
    """
    visibility: ClassVar[Sequence[str]] = ['public']
    
    hour: int = 0  # 0-23
    
    def vector(self) -> np.ndarray:
        """Encode cyclically (hour 23 is close to hour 0)."""
        rad = 2 * np.pi * self.hour / 24
        return np.array([np.sin(rad), np.cos(rad)], dtype=np.float32)
    
    def names(self):
        return ['hour_sin', 'hour_cos']
    
    def set_values(self, **kw):
        if 'hour' in kw:
            self.hour = int(kw['hour']) % 24


# Test
time = TimeOfDay(hour=14)  # 2 PM
print(f"Time vector: {time.vector()}")
print(f"Visibility: {time.visibility}")

Time vector: [-0.5       -0.8660254]
Visibility: ['public']


## How Visibility Filtering Works

HERON automatically checks visibility based on agent hierarchy using `is_observable_by()`:

In [4]:
# BatterySOC has visibility = ['owner', 'upper_level']
battery_soc = BatterySOC(soc=0.8)

# Check who can see this feature:

# 1. Owner (the battery itself) - YES
can_owner_see = battery_soc.is_observable_by(
    requestor_id='battery_1', requestor_level=1,
    owner_id='battery_1', owner_level=1
)
print(f"Battery can see its own SOC: {can_owner_see}")  # True

# 2. Coordinator (upper_level, level 2) - YES  
can_coord_see = battery_soc.is_observable_by(
    requestor_id='microgrid_1', requestor_level=2,  # One level above
    owner_id='battery_1', owner_level=1
)
print(f"Coordinator can see battery SOC: {can_coord_see}")  # True

# 3. Another battery (peer, not upper_level) - NO
can_peer_see = battery_soc.is_observable_by(
    requestor_id='battery_2', requestor_level=1,  # Same level, not owner
    owner_id='battery_1', owner_level=1
)
print(f"Other battery can see SOC: {can_peer_see}")  # False

Battery can see its own SOC: True
Coordinator can see battery SOC: True
Other battery can see SOC: False


## State Classes for All Hierarchy Levels

HERON provides three state classes matching the 3-level agent hierarchy:

| Level | Agent Type | State Class | Typical Features |
|-------|------------|-------------|------------------|
| L1 | FieldAgent | `FieldAgentState` | Device state (SOC, power output) |
| L2 | CoordinatorAgent | `CoordinatorAgentState` | Aggregated metrics, local load |
| L3 | SystemAgent | `SystemAgentState` | System-wide metrics (frequency, total generation) |

**Key Pattern:** Features are stored as a dict keyed by `feature_name`:
```python
state = FieldAgentState(
    owner_id='battery_1',
    owner_level=1,
    features={f.feature_name: f for f in [BatterySOC(soc=0.7)]}
)
```

In [5]:
from heron.core.state import FieldAgentState, CoordinatorAgentState, SystemAgentState

# ===== L1: FieldAgentState (device level) =====
# Create state for a battery agent with features as dict
battery_feature = BatterySOC(soc=0.7)
battery_state = FieldAgentState(
    owner_id='battery_1',
    owner_level=1,
    features={battery_feature.feature_name: battery_feature}
)

# Get full observation vector (all features)
full_obs = battery_state.vector()
print(f"Full state vector: {full_obs}")

# Get observation filtered by requestor
filtered_obs = battery_state.observed_by(requestor_id='battery_1', requestor_level=1)
print(f"Filtered observation: {filtered_obs}")


# ===== L2: CoordinatorAgentState (coordinator level) =====
@dataclass(slots=True)
class AggregatedLoad(FeatureProvider):
    """Total load across all subordinate devices."""
    visibility: ClassVar[Sequence[str]] = ['owner', 'system']
    
    total_mw: float = 0.0
    
    def vector(self) -> np.ndarray:
        return np.array([self.total_mw / 100.0], dtype=np.float32)  # Normalize

coord_feature = AggregatedLoad(total_mw=45.0)
coord_state = CoordinatorAgentState(
    owner_id='microgrid_1',
    owner_level=2,
    features={coord_feature.feature_name: coord_feature}
)
print(f"CoordinatorAgentState vector: {coord_state.vector()}")


# ===== L3: SystemAgentState (system level) =====
@dataclass(slots=True)
class SystemFrequency(FeatureProvider):
    """Grid frequency - system-wide metric."""
    visibility: ClassVar[Sequence[str]] = ['system']
    
    frequency_hz: float = 60.0
    nominal_hz: float = 60.0
    
    def vector(self) -> np.ndarray:
        deviation = (self.frequency_hz - self.nominal_hz) / self.nominal_hz
        return np.array([deviation], dtype=np.float32)

sys_feature = SystemFrequency(frequency_hz=59.95)
system_state = SystemAgentState(
    owner_id='grid_system',
    owner_level=3,
    features={sys_feature.feature_name: sys_feature}
)
print(f"SystemAgentState vector: {system_state.vector()}")

# All three state types use the same API pattern!

Full state vector: [0.7]
Filtered observation: {'BatterySOC': array([0.7], dtype=float32)}
CoordinatorAgentState vector: [0.45]
SystemAgentState vector: [-0.00083333]


## Feature Auto-Registration

HERON uses a metaclass to automatically register all FeatureProvider subclasses. This enables `State.from_dict()` to reconstruct features by name:

In [6]:
from heron.core.feature import get_all_registered_features, get_feature_class

# See all registered features
registered = get_all_registered_features()
print(f"Number of registered features: {len(registered)}")
print(f"Some registered features: {list(registered.keys())[:5]}...")

# Get a specific feature class by name
BatterySOCClass = get_feature_class('BatterySOC')
new_battery = BatterySOCClass(soc=0.3)
print(f"\nCreated from registry: {new_battery}")

Number of registered features: 5
Some registered features: ['BatterySOC', 'GenOutput', 'TimeOfDay', 'AggregatedLoad', 'SystemFrequency']...

Created from registry: BatterySOC(soc=0.3)


## Design Guidelines

| Feature Type | Visibility | Example |
|--------------|------------|---------|
| Private device state | `['owner']` | Internal temperature |
| Hierarchical info | `['owner', 'upper_level']` | Battery SOC, generator output |
| System metrics | `['owner', 'upper_level', 'system']` | Total generation |
| Public info | `['public']` | Time, weather, prices |

**Tips:**
- **Normalize in `vector()`**: ML algorithms prefer [0, 1] or [-1, 1]
- **Store raw values**: Keep originals in fields for debugging/serialization
- **Use `@dataclass(slots=True)`**: Memory efficient, clean definitions
- **Use `ClassVar` for visibility**: Required by the type system
- **Override only when needed**: Base class provides sensible defaults

## Summary: Features We Built

```python
# L1 (Field) Features
BatterySOC      →  visibility = ['owner', 'upper_level']
GenOutput       →  visibility = ['owner', 'upper_level', 'system']
TimeOfDay       →  visibility = ['public']

# L2 (Coordinator) Features
AggregatedLoad  →  visibility = ['owner', 'system']

# L3 (System) Features
SystemFrequency →  visibility = ['system']
```

These features will be used by agents in the next tutorial.

In [7]:
# Verify our features work
print("Features defined successfully!")
print(f"  BatterySOC: {BatterySOC(soc=0.5).vector()}")
print(f"  GenOutput: {GenOutput(p_mw=2.5, p_max=5.0).vector()}")
print(f"  TimeOfDay: {TimeOfDay(hour=12).vector()}")

Features defined successfully!
  BatterySOC: [0.5]
  GenOutput: [0.5]
  TimeOfDay: [ 1.2246469e-16 -1.0000000e+00]


## Key Takeaways

1. **HERON is agent-centric** — Agents have state, timing, hierarchy (not just policy wrappers)
2. **Visibility is declarative** — Just set `visibility: ClassVar = ['owner', 'upper_level']`
3. **No manual filtering** — HERON handles observation filtering automatically via `is_observable_by()`
4. **Three state classes for three hierarchy levels**:
   - `FieldAgentState` (L1) — Device-level state
   - `CoordinatorAgentState` (L2) — Coordinator-level aggregated metrics
   - `SystemAgentState` (L3) — System-wide metrics
5. **Features stored as dict** — `{feature_name: feature}` for easy lookup
6. **Use `@dataclass(slots=True)`** — Memory efficient with auto-generated methods
7. **Override only when needed** — Base class provides `vector()`, `names()`, serialization

---

**Next:** [02_building_agents.ipynb](02_building_agents.ipynb) — Create agents that use these features